In [ ]:
###############
## Libraries ##
###############

import boto3
import pandas as pd
import numpy as np
import outlier_analysis
import Book_Recommendation
import kNN

s3 = boto3.resource('s3')
bucket_name = 'dmc-2021'
bucket = s3.Bucket(bucket_name)

## Defining files names
file_key_1 = 'items_final_newer.csv'
file_key_2 = 'transactions.csv'
file_key_3 = 'evaluation.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading csv form s3 buckets
items = pd.read_csv(file_content_stream_1)
transactions = pd.read_csv(file_content_stream_2, sep = '|')
evaluation  = pd.read_csv(file_content_stream_3)

## Counting number of clicks, basket and sales
click = transactions.groupby('itemID')['click'].sum().reset_index()
click_count = transactions.groupby('itemID')['click'].count().reset_index()
click_count.columns = ['itemID', 'click_count']

basket = transactions.groupby('itemID')['basket'].sum().reset_index()
basket_count = transactions.groupby('itemID')['basket'].count().reset_index()
basket_count.columns = ['itemID', 'basket_count']

orders = transactions.groupby('itemID')['order'].sum().reset_index()
orders_count = transactions.groupby('itemID')['order'].count().reset_index()
orders_count.columns = ['itemID', 'orders_count']

## Appending clicks, basket, and orders
items_transactions = pd.DataFrame(items['itemID'])

items_transactions = pd.merge(items_transactions, click, how = 'left', on = 'itemID')
items_transactions = pd.merge(items_transactions, click_count, how = 'left', on = 'itemID')

items_transactions = pd.merge(items_transactions, basket, how = 'left', on = 'itemID')
items_transactions = pd.merge(items_transactions, basket_count, how = 'left', on = 'itemID')

items_transactions = pd.merge(items_transactions, orders, how = 'left', on = 'itemID')
items_transactions = pd.merge(items_transactions, orders_count, how = 'left', on = 'itemID')

## Computing item ratings
item_ratings = outlier_analysis.distance_from_centroid_scores(items_transactions)
item_ratings = item_ratings[['itemID', 'rating']]

## Running book recommendation engine (approach 2)
similarity_measure = 'Euclidean'
recommendation_2 = Book_Recommendation.Book_Recommendation(items, transactions, evaluation, item_ratings, similarity_measure)